This notebook uses the `InteractionEncoder` to replicate the regression output from an economics paper by **Frankel & Romer (1999)**: [Does Trade Cause Growth?](https://www.aeaweb.org/articles?id=10.1257/aer.89.3.379) (paper aer.89.3.379).  The data files are sourced from that webpage.  The `tablelist` folder contains the relevant data sources and files that specify how to produce the models from the paper.

**Table 1 from the paper is replicated in this notebook.**  The model finds that the presence of a common border (variable `adj`) between two countries (`i` & `j`) changes the effects of other factors on the amount of trade that country `i` does with country `j`.

The table has the interactions between `adj` (the common border variable) with many other geographical variables that influence trade, e.g. distance between countries and size of countries, so it is a nice example of how the Appelpy encoder can produce interaction effects relatively easily.

In [1]:
import pandas as pd
import numpy as np

# Appelpy imports
from appelpy.utils import InteractionEncoder
from appelpy.eda import statistical_moments
from appelpy.linear_model import OLS

# Load data

The data files are from an old version of Stata, which is not supported by the Pandas `read_stata` function.  They can be read by the `read_csv` method, via some arguments and pre-processing.

There are multiple datasets to be combined into one dataframe.  They are in the `tablelist` folder from the data sources provided on the American Economic Association webpage.

Each dataset has 3782 observations and a common key, so the files can be simply concatenated.

In [2]:
trade62 = pd.read_csv('data/frankel-romer_tablelist/trade62.dat', delim_whitespace=True, header=None)
trade62.columns = ['codesj', 'trade']
trade62.tail()

,codesj,trade
3777,6459,405.0
3778,6460,1512.0
3779,6461,685.0
3780,6462,166.0
3781,6463,522.0


In [3]:
dist62 = pd.read_csv('data/frankel-romer_tablelist/dist62.dat', delim_whitespace=True, header=None)
dist62.columns = ['codesj', 'dist', 'adj']
dist62.tail()

,codesj,dist,adj
3777,6459,1840.548,0
3778,6460,3798.440,0
3779,6461,2876.819,0
3780,6462,8353.153,0
3781,6463,7978.150,0


In [4]:
var62i = pd.read_csv('data/frankel-romer_tablelist/var62i.dat', delim_whitespace=True, header=None)
var62i.columns = ['codei', 'gdpi', 'locki', 'worki', 'areai']
var62i.tail()

,codei,gdpi,locki,worki,areai
3777,62.0,1.326380e+09,0.0,612363.1,3689631.0
3778,62.0,1.326380e+09,0.0,612363.1,3689631.0
3779,62.0,1.326380e+09,0.0,612363.1,3689631.0
3780,62.0,1.326380e+09,0.0,612363.1,3689631.0
3781,62.0,1.326380e+09,0.0,612363.1,3689631.0


In [5]:
var62j = pd.read_csv('data/frankel-romer_tablelist/var62j.dat', delim_whitespace=True, header=None)
var62j.columns = ['codej', 'gdpj', 'lockj', 'workj', 'areaj']

var62j['gdpj'] = var62j['gdpj'].str.replace('D', 'E').astype(float)

var62j.tail()

,codej,gdpj,lockj,workj,areaj
3777,57,84347400.0,0,19944.99805,115830
3778,58,21393500.0,0,1189.45447,220
3779,59,127295000.0,0,26793.35547,198455
3780,60,56247600.0,1,5195.12744,35920
3781,61,155397000.0,0,19234.67383,120728


In [6]:
df_pre = pd.concat([trade62, dist62.drop(columns=['codesj']), var62i, var62j], axis='columns')
df_pre.head()

,codesj,trade,dist,adj,codei,gdpi,locki,worki,areai,codej,gdpj,lockj,workj,areaj
0,102,1610.0,5652.806,0,1.0,392297000.0,0.0,12595.02,3851809.0,2,6.734050e+08,0,24881.94727,211208
1,103,2761.0,5856.608,0,1.0,392297000.0,0.0,12595.02,3851809.0,3,7.653620e+08,0,28084.61914,96010
2,104,1382.0,6735.185,0,1.0,392297000.0,0.0,12595.02,3851809.0,4,6.175800e+08,0,22714.33008,116500
3,105,8875.0,10327.390,0,1.0,392297000.0,0.0,12595.02,3851809.0,5,1.421400e+09,0,75525.78906,143574
4,106,3831.0,5367.578,0,1.0,392297000.0,0.0,12595.02,3851809.0,6,6.362160e+08,0,27684.45508,94247


# Initial feature engineering

Create a copy of the dataframe.

More feature engineering will be done before modelling.

In [7]:
df = df_pre.copy()

**The dependent variable for the model is `ltrdsh`.**

That is the logarithm of country `i`'s trade with country `j` as a proportion of country `i`'s GDP.

In [8]:
df['trdsh'] = df['trade'] / df['gdpi']
df['ltrdsh'] = np.where(df['trdsh'] == 0, np.NaN, np.log(df['trdsh']))

C:\ProgramData\Miniconda3\envs\appelpy\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


Log transformation of independent variables.

In [9]:
# Log transformations of many variables
df['lworki'] = np.log(df['worki'])
df['lworkj'] = np.log(df['workj'])
df['lareai'] = np.log(df['areai'])
df['lareaj'] = np.log(df['areaj'])
df['ldist'] = np.log(df['dist'])

In [10]:
# Variable to represent how many of the countries i & j are landlocked (between 0 and 2)
df['lockij'] = df['locki'] + df['lockj']

In [11]:
statistical_moments(df)

,mean,var,skew,kurtosis
codesj,3279.24,3.50782e+06,0.00453871,-1.2432
trade,813.376,1.94175e+07,17.6183,413.533
dist,7922.44,2.27926e+07,0.353816,-0.567253
adj,0.0333157,0.0322058,5.20099,25.0503
codei,31.5,320.25,0,-1.20062
gdpi,2.55534e+08,3.08501e+17,5.08283,29.6711
locki,0.0806452,0.0741415,3.08021,7.48772
worki,28243.5,7.22723e+09,5.74072,34.5483
areai,516431,8.53008e+11,2.66406,5.99213
codej,31.5,320.25,0,-1.20062


# Model with interaction terms

## Via `InteractionEncoder`

Set up the encoder object.

Encode the interaction effects and store them in `df_model`.

In [12]:
int_encoder = InteractionEncoder(df)

In [13]:
df_model = int_encoder.encode({'adj': ['lockj', 'ldist', 'lockij', 'lworki', 'lworkj', 'lareai', 'lareaj']})

In [14]:
print(f"Columns added: {int_encoder.columns_added}")

Columns added: ['adj#lockij', 'adj#lockj', 'adj#ldist', 'adj#lworkj', 'adj#lareai', 'adj#lworki', 'adj#lareaj']


The encoder uses the nullable integer dtype (introduced in Pandas 0.24) for producing dummy columns.  Notice how `adj#lockj` is `Int64` (i.e. `pd.Int64Dtype()`.

However nullable integer dtype is not supported in Statsmodels regression objects, so those will be converted to `int64`.  After all, dataframes used in Statsmodels regression must not have any null values.

In [15]:
df_model.dtypes.loc[['adj', 'lockj', 'adj#lockj']]

adj          int64
lockj        int64
adj#lockj    Int64
dtype: object

In [16]:
for col in df_model.select_dtypes(pd.Int64Dtype()):
    df_model[col] = df_model[col].astype('int64')

In [17]:
df_model.head(10)

,codesj,trade,dist,adj,codei,gdpi,locki,worki,areai,codej,...,lareaj,ldist,lockij,adj#lockj,adj#ldist,adj#lockij,adj#lworki,adj#lworkj,adj#lareai,adj#lareaj
0,102,1610.0,5652.806,0,1.0,392297000.0,0.0,12595.02,3851809.0,2,...,12.260599,8.639907,0.0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,103,2761.0,5856.608,0,1.0,392297000.0,0.0,12595.02,3851809.0,3,...,11.472208,8.675326,0.0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,104,1382.0,6735.185,0,1.0,392297000.0,0.0,12595.02,3851809.0,4,...,11.665647,8.815101,0.0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,105,8875.0,10327.390,0,1.0,392297000.0,0.0,12595.02,3851809.0,5,...,11.874606,9.242555,0.0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,106,3831.0,5367.578,0,1.0,392297000.0,0.0,12595.02,3851809.0,6,...,11.453674,8.588132,0.0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
5,107,122257.0,1037.087,1,1.0,392297000.0,0.0,12595.02,3851809.0,7,...,15.079903,6.944171,0.0,0,6.944171,0.0,9.441057,11.673022,15.164053,15.079903
6,108,187.0,6573.839,0,1.0,392297000.0,0.0,12595.02,3851809.0,8,...,10.385142,8.790853,1.0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,109,834.0,5679.494,0,1.0,392297000.0,0.0,12595.02,3851809.0,9,...,9.374243,8.644617,0.0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
8,110,223.0,5913.385,0,1.0,392297000.0,0.0,12595.02,3851809.0,10,...,9.719024,8.684974,0.0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,111,214.0,6278.729,0,1.0,392297000.0,0.0,12595.02,3851809.0,11,...,11.776205,8.744923,0.0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


Gather the independent variables in a list `X_cols` and fit a regression model.

In [18]:
X_cols = ['ldist', 'lworki', 'lareai', 'lworkj', 'lareaj', 'lockij', 
          'adj', 'adj#ldist', 'adj#lworki', 'adj#lareai', 'adj#lworkj', 'adj#lareaj', 'adj#lockij']

In [19]:
model = OLS(df_model, ['ltrdsh'], X_cols)

C:\ProgramData\Miniconda3\envs\appelpy\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Model fitting in progress...
Model fitted.


In [20]:
model.results_output

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ltrdsh   R-squared:                       0.361
Model:                            OLS   Adj. R-squared:                  0.358
Method:                 Least Squares   F-statistic:                     139.1
Date:                Sun, 07 Apr 2019   Prob (F-statistic):          2.05e-299
Time:                        22:58:58   Log-Likelihood:                -6161.7
No. Observations:                3220   AIC:                         1.235e+04
Df Residuals:                    3206   BIC:                         1.244e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.3777      0.415    -15.361      0.000      -7.192      -5.564
ldist         -0.8521      0.038    -22.653      0.000      -0.926      -0.778
lworki        -0.2393      0.026     -9.326      0.000      -0.290      -0.189
lareai        -0.1206      0.018     -6.588      0.000      -0.156      -0.085
lworkj         0.6093      0.026     23.745      0.000       0.559       0.660
lareaj        -0.1851      0.018    -10.114      0.000      -0.221      -0.149
lockij        -0.3609      0.084     -4.314      0.000      -0.525      -0.197
adj            5.0952      1.780      2.863      0.004       1.606       8.585
adj#ldist      0.1507      0.303      0.498      0.619      -0.443       0.744
adj#lworki    -0.2924      0.181     -1.619      0.106      -0.646       0.062
adj#lareai    -0.0572      0.151     -0.378      0.705      -0.354       0.240
adj#lworkj    -0.1365      0.181     -0.756      0.450      -0.491       0.218
adj#lareaj    -0.0691      0.151     -0.456      0.648      -0.366       0.228
adj#lockij     0.3262      0.331      0.986      0.324      -0.322       0.975
==============================================================================
Omnibus:                       48.172   Durbin-Watson:                   1.153
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.111
Skew:                          -0.306   Prob(JB):                     1.31e-11
Kurtosis:                       3.000   Cond. No.                     1.41e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.41e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**The output above replicates Table 1 in the paper.**

This is one of the key sentences in the paper about the table:

"The estimates also imply that the presence of a common border alters the effects of the other variables substantially."

## Via manually coded interactions

This section does manual coding of the interaction effects.  The variable names, e.g. `adjlij`, are all from the data files used by the paper's authors.

Even with simpler approaches with less code, e.g. loops, there is scope for error.  The `InteractionEncoder` is more practical for a pipeline of interaction effect generation.

In [21]:
df_model['adjdist'] = df_model['adj'] * df_model['ldist']
df_model['adjlij'] = df_model['adj'] * df_model['lockij']
df_model['adjworki'] = df_model['adj'] * df_model['lworki']
df_model['adjworkj'] = df_model['adj'] * df_model['lworkj']
df_model['adjareai'] = df_model['adj'] * df_model['lareai']
df_model['adjareaj'] = df_model['adj'] * df_model['lareaj']

In [22]:
X_cols_manual = ['ldist', 'lworki', 'lareai', 'lworkj', 'lareaj', 'lockij', 
                 'adj', 'adjdist', 'adjworki', 'adjareai', 'adjworkj', 'adjareaj', 'adjlij']

In [23]:
model_manual = OLS(df_model, ['ltrdsh'], X_cols_manual)

Model fitting in progress...
Model fitted.


C:\ProgramData\Miniconda3\envs\appelpy\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [24]:
model_manual.results_output

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ltrdsh   R-squared:                       0.361
Model:                            OLS   Adj. R-squared:                  0.358
Method:                 Least Squares   F-statistic:                     139.1
Date:                Sun, 07 Apr 2019   Prob (F-statistic):          2.05e-299
Time:                        22:58:59   Log-Likelihood:                -6161.7
No. Observations:                3220   AIC:                         1.235e+04
Df Residuals:                    3206   BIC:                         1.244e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.3777      0.415    -15.361      0.000      -7.192      -5.564
ldist         -0.8521      0.038    -22.653      0.000      -0.926      -0.778
lworki        -0.2393      0.026     -9.326      0.000      -0.290      -0.189
lareai        -0.1206      0.018     -6.588      0.000      -0.156      -0.085
lworkj         0.6093      0.026     23.745      0.000       0.559       0.660
lareaj        -0.1851      0.018    -10.114      0.000      -0.221      -0.149
lockij        -0.3609      0.084     -4.314      0.000      -0.525      -0.197
adj            5.0952      1.780      2.863      0.004       1.606       8.585
adjdist        0.1507      0.303      0.498      0.619      -0.443       0.744
adjworki      -0.2924      0.181     -1.619      0.106      -0.646       0.062
adjareai      -0.0572      0.151     -0.378      0.705      -0.354       0.240
adjworkj      -0.1365      0.181     -0.756      0.450      -0.491       0.218
adjareaj      -0.0691      0.151     -0.456      0.648      -0.366       0.228
adjlij         0.3262      0.331      0.986      0.324      -0.322       0.975
==============================================================================
Omnibus:                       48.172   Durbin-Watson:                   1.153
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.111
Skew:                          -0.306   Prob(JB):                     1.31e-11
Kurtosis:                       3.000   Cond. No.                     1.41e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.41e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""